# SWOW Loader — Smoke Stub
Author: Demetrios Agourakis (ORCID 0000-0002-8596-5097)
Concept DOI: 10.5281/zenodo.16921951 — Version DOI: 10.5281/zenodo.17053446
License: Code MIT; Text CC BY 4.0


In [ ]:
# Deterministic environment & smoke output (no external deps)
import os, sys, json, platform, random
random.seed(42)
os.makedirs('data/processed', exist_ok=True)
out = {"ok": True, "notebook": "01_swow_loader", "python": sys.version.split()[0]}
with open('data/processed/_stub_swow.json','w') as f: json.dump(out, f)
print('stub written:', 'data/processed/_stub_swow.json')


In [ ]:
# Provenance & Environment
print('Python', platform.python_version())
print('Platform', platform.platform())
